# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
vacay = pd.read_csv("output_data_file.csv", encoding="utf-8")
vacay.reset_index()

,index,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Temp,Wind Speed
0,0,0,Faanui,100,PF,1644773009,73,-16.4833,-151.7500,81.07,14.32
1,1,1,Bethel,100,US,1644773009,79,41.3712,-73.4140,31.77,4.61
2,2,2,Saldanha,19,ZA,1644773010,85,-33.0117,17.9442,75.25,27.67
3,3,3,Sweetwater,100,US,1644773010,88,25.7634,-80.3731,66.22,6.91
4,4,4,Busselton,4,AU,1644773010,68,-33.6500,115.3333,65.05,12.84
...,...,...,...,...,...,...,...,...,...,...,...
552,552,552,Barcelos,92,PT,1644773172,90,41.5388,-8.6151,57.34,12.48
553,553,553,Waipawa,100,NZ,1644773173,84,-41.4122,175.5153,54.23,11.74
554,554,554,San Julián,93,MX,1644773173,50,21.0167,-102.1667,57.49,10.04
555,555,555,Luganville,96,VU,1644773174,85,-15.5333,167.1667,78.57,4.92


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)

In [5]:
locations = vacay[["Lat", "Lng"]].astype(float)
print(locations)

humidity = vacay["Humidity"].astype(float)

         Lat       Lng
0   -16.4833 -151.7500
1    41.3712  -73.4140
2   -33.0117   17.9442
3    25.7634  -80.3731
4   -33.6500  115.3333
..       ...       ...
552  41.5388   -8.6151
553 -41.4122  175.5153
554  21.0167 -102.1667
555 -15.5333  167.1667
556  50.6833   10.9000

[557 rows x 2 columns]


In [6]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
#my ideal temperature for a vacation is pretty different than the example!

#A max temperature between 80 and 100 degrees
ideal_vacay = vacay[vacay["Temp"].between(80,100, inclusive=True)]

#Humidity greater than or equal to 50%
ideal_vacay = ideal_vacay[ideal_vacay["Humidity"] >=50]

#low cloudiness
ideal_vacay = ideal_vacay[ideal_vacay["Cloudiness"] <15]

#there was a city with a wind speed of 13... I'm not superstitious, but I am a little stitious
ideal_vacay = ideal_vacay[ideal_vacay["Wind Speed"] <13]

ideal_vacay.dropna(how='any')
ideal_vacay.reset_index(inplace=True)
del ideal_vacay['index']
ideal_vacay

/var/folders/gr/g0lzn1p15gqfb11105yyt_tm0000gn/T/ipykernel_662/3008660479.py:4: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated infavour of `both` or `neither`.
  ideal_vacay = vacay[vacay["Temp"].between(80,100, inclusive=True)]


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Temp,Wind Speed
0,112,Ancud,5,CL,1644773040,73,-41.8697,-73.8203,84.22,7.70
1,123,Cabedelo,3,BR,1644773043,65,-6.9811,-34.8339,88.47,10.58
2,151,Castro,11,BR,1644773052,51,-24.7911,-50.0119,80.26,5.28
3,178,Lima,0,PE,1644772280,62,-12.0432,-77.0282,81.90,7.00
4,189,Cananéia,5,BR,1644773064,55,-25.0147,-47.9267,83.07,9.08
5,373,Touros,12,BR,1644773119,61,-5.1989,-35.4608,88.70,12.55
6,382,Morant Bay,4,JM,1644773122,63,17.8815,-76.4093,85.62,12.57
7,420,Kavaratti,5,IN,1644773132,68,10.5669,72.6420,81.27,5.17
8,427,Lucea,5,JM,1644773135,63,18.4510,-78.1736,82.26,11.90
9,428,Puerto El Triunfo,8,SV,1644773135,50,13.2833,-88.5500,92.26,7.78


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = ideal_vacay
hotel_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Temp,Wind Speed
0,112,Ancud,5,CL,1644773040,73,-41.8697,-73.8203,84.22,7.70
1,123,Cabedelo,3,BR,1644773043,65,-6.9811,-34.8339,88.47,10.58
2,151,Castro,11,BR,1644773052,51,-24.7911,-50.0119,80.26,5.28
3,178,Lima,0,PE,1644772280,62,-12.0432,-77.0282,81.90,7.00
4,189,Cananéia,5,BR,1644773064,55,-25.0147,-47.9267,83.07,9.08
5,373,Touros,12,BR,1644773119,61,-5.1989,-35.4608,88.70,12.55
6,382,Morant Bay,4,JM,1644773122,63,17.8815,-76.4093,85.62,12.57
7,420,Kavaratti,5,IN,1644773132,68,10.5669,72.6420,81.27,5.17
8,427,Lucea,5,JM,1644773135,63,18.4510,-78.1736,82.26,11.90
9,428,Puerto El Triunfo,8,SV,1644773135,50,13.2833,-88.5500,92.26,7.78


In [9]:
hotel_df["Hotel Name"] = ""
hotel_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Temp,Wind Speed,Hotel Name
0,112,Ancud,5,CL,1644773040,73,-41.8697,-73.8203,84.22,7.70,
1,123,Cabedelo,3,BR,1644773043,65,-6.9811,-34.8339,88.47,10.58,
2,151,Castro,11,BR,1644773052,51,-24.7911,-50.0119,80.26,5.28,
3,178,Lima,0,PE,1644772280,62,-12.0432,-77.0282,81.90,7.00,
4,189,Cananéia,5,BR,1644773064,55,-25.0147,-47.9267,83.07,9.08,
5,373,Touros,12,BR,1644773119,61,-5.1989,-35.4608,88.70,12.55,
6,382,Morant Bay,4,JM,1644773122,63,17.8815,-76.4093,85.62,12.57,
7,420,Kavaratti,5,IN,1644773132,68,10.5669,72.6420,81.27,5.17,
8,427,Lucea,5,JM,1644773135,63,18.4510,-78.1736,82.26,11.90,
9,428,Puerto El Triunfo,8,SV,1644773135,50,13.2833,-88.5500,92.26,7.78,


In [16]:
for index, row in hotel_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    lat = row['Lat']
    lng = row['Lng']
    
    try:
        params = {
            'location': f"{lat},{lng}",
            'keyword': 'hotel',
            'radius': 5000,
            'key': g_key
        }
        
        hotel_data = requests.get(base_url, params=params).json()
        
        hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
        
    except IndexError:
        
        hotel_df.loc[index, "Hotel Name"] = "NaN"
        
hotel_df       

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Temp,Wind Speed,Hotel Name
0,112,Ancud,5,CL,1644773040,73,-41.8697,-73.8203,84.22,7.70,Panamericana Hotel Ancud
1,123,Cabedelo,3,BR,1644773043,65,-6.9811,-34.8339,88.47,10.58,NaN
2,151,Castro,11,BR,1644773052,51,-24.7911,-50.0119,80.26,5.28,Central Palace Hotel
3,178,Lima,0,PE,1644772280,62,-12.0432,-77.0282,81.90,7.00,Gran Hotel Bolivar Lima
4,189,Cananéia,5,BR,1644773064,55,-25.0147,-47.9267,83.07,9.08,Hotel Marazul
5,373,Touros,12,BR,1644773119,61,-5.1989,-35.4608,88.70,12.55,Hotel Vila Gale Touros
6,382,Morant Bay,4,JM,1644773122,63,17.8815,-76.4093,85.62,12.57,Golden Shore Resort
7,420,Kavaratti,5,IN,1644773132,68,10.5669,72.6420,81.27,5.17,Bismillah Hotel And Lodging
8,427,Lucea,5,JM,1644773135,63,18.4510,-78.1736,82.26,11.90,Grand Palladium Lady Hamilton Resort & Spa
9,428,Puerto El Triunfo,8,SV,1644773135,50,13.2833,-88.5500,92.26,7.78,NaN


In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [19]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig
# Display figure


Figure(layout=FigureLayout(height='420px'))